# Scraping the Cordis website

I'm not sure yet about legality of this. The website's legal information only contains information about how th users data is used, not how the user is allowed to interact with the data from the website. The only thing it mentioned is that the content is copyright protected and needs to be cited properly.

## Help Files
- [Introduction to web scraping with selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os

- 1st import: Allows you to launch/initialise a browser.
- 2nd import: Allows you to search for things using specific parameters.
- 3rd import: Allows you to wait for a page to load.
- 4th import: Specify what you are looking for on a specific page in order to determine that the webpage has loaded.
- 5th import: Handling a timeout situation.

## Creating helper function

create new instance of chrome in incognito mode

In [2]:
option = webdriver.ChromeOptions()
option.add_argument("— incognito")

In [3]:
#chromedriver_path = "/Users/guysimons/Documents/BISS/FuturizingBusinessSchools/Scraping Python/Drivers/chromedriver"
firefoxdriver_path = '/Users/katerinadoyle/Documents/java/webscraping/drivers/geckodriver'

In [80]:
# change the xpath 
def init_driver():
    driver = webdriver.Firefox(executable_path=firefoxdriver_path) #changed this. Chrome takes too much CPU on my mac
    #driver.wait = WebDriverWait(driver, 5)
    #driver.implicitly_wait(10)
    return driver

def navigate_to_page(driver, homepage):
     driver.get(homepage)
     time.sleep(10)
     
def navigate_to_keyword_page(driver, keyword, homepage):
     driver.get(homepage)
     time.sleep(10)
     inputElement = driver.find_element_by_id("projectsResultsSearchValue") # looks for the search field
     inputElement.send_keys(keyword) # enters the keyword
     inputElement.send_keys(Keys.ENTER) # simulate hitting enter
     time.sleep(10) # wait for page to load
     select = Select(driver.find_element_by_name("num"))
     #print ([o.text for o in select.options]) # testing purpose: what options are available
     select.select_by_value("100")
     time.sleep(10)

def get_project_url(driver):
     # go to report summary page
     project_page_html = driver.find_elements_by_xpath("//a[contains(@href,'result/rcn')]")
     return project_page_html

def get_all_project_content(driver):
    # two options: scrape content from site OR download pdf/xml/rtf file. XML file might be the best
     #prj_id = driver.find_element_by_xpath("//a[@class]")
     #funded_under = driver.find_elemnt_by_xpath
     country = driver.find_element_by_xpath("//div[@class = 'country']")
     title = driver.find_element_by_xpath("//div[@class = 'content']/h2")
     content = driver.find_element_by_xpath("//div[@class = 'article']")
     subjects = driver.find_element_by_xpath("//div[@class = 'subject']")
     return country
     return title
     return content
     return subjects

def get_project_content(driver):
     content = driver.find_element_by_xpath("//div[@class = 'article']")
     return content
    
def next_page(driver):
     nxt_button = driver.find_element_by_xpath("//span[@data-tn-element='next-page']")
     time.sleep(2)
     nxt_button.click()
     time.sleep(2)

## Accessing the webpage and getting the data

In [65]:
homepage= 'https://cordis.europa.eu/projects/home_en.html'

In [66]:
driver = init_driver()
#time.sleep(20)

In [67]:
# go to cordis page, enter the keyword, display results, and set search parameters to 'showing 100 results per page'
navigate_to_keyword_page(driver, 'neurological AND disorder', homepage)

In [81]:
# collect the project urls and store them in a list
projecturls = []
projecturls = get_project_url(driver)
# modify it now to loop through all search pages. 

In [69]:
print(len(projecturls))
print(projecturls[0].text)

136
Final Report Summary - ADDUCE (Attention Deficit Hyperactivity Disorder Drugs Use Chronic Effects)


It got 136 entries. The text attribute returns the anchor text. To get the href attribute, type 'get_attribute("href")

In [70]:
for link in projecturls[0:1]:
    print(link.get_attribute("href"))

https://cordis.europa.eu/result/rcn/187956_en.html


In [84]:
with open('cordis_project_link.txt', 'w') as file:
    for link in projecturls:
        url = link.get_attribute("href")
        file.write(url+'\n')

In [85]:
#import os
#print(os.listdir())

['.DS_Store', '.git', '.gitignore', '.ipynb_checkpoints', 'chromedriver', 'cordis_project_link.txt', 'geckodriver', 'geckodriver.log', 'neurological_disorders.md', 'neurology.csv', 'notes.md', 'README.md', 'scrape_ckan_api.ipynb', 'scrape_cordis.ipynb', 'scraper.py', 'test.txt']


In [97]:
content=[]
with open('cordis_project_link.txt', 'r') as urlfile:
    for line in urlfile:
        #print(line)
        navigate_to_page(driver, line)
        text = get_project_content(driver)
        content.append(text.text)
        time.sleep(10)

In [99]:
with open('cordis_project_content.txt', 'w') as file:
    for item in content:
        file.write(item+'\n')

In [108]:
import io
i = 0
for i in range(0,len(content)):
    with io.open("cordis_doc/file_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(content[i])
    i=+1

Try to see if we can get extended content and save it as one document

In [109]:
content={ }
with open('cordis_project_link.txt', 'r') as urlfile:
    for line in urlfile:
        #print(line)
        navigate_to_page(driver, line)
        text = get_all_project_content(driver)
        content.append(text.text)
        time.sleep(10)

AttributeError: 'dict' object has no attribute 'append'